In [1]:
from openeye import oechem, oespruce

# Loading protein, splitting, and aligning to reference

In [10]:
pdb_fn = "input_pdbs/5WKK.pdb"
## Load molecule to extract
ifs = oechem.oemolistream()
ifs.SetFlavor(
    oechem.OEFormat_PDB,
    oechem.OEIFlavor_PDB_Default
    | oechem.OEIFlavor_PDB_DATA
    | oechem.OEIFlavor_PDB_ALTLOC,
)
ifs.open(pdb_fn)
complex_mol = oechem.OEGraphMol()
oechem.OEReadMolecule(ifs, complex_mol)
ifs.close()

## Keep only highest occupancy alt location
alf = oechem.OEAltLocationFactory(complex_mol)
if alf.GetGroupCount() != 0:
    print(complex_mol.NumAtoms())
    alf.MakePrimaryAltMol(complex_mol)
    print(complex_mol.NumAtoms())

2634
2603


In [14]:
## Test splitting
lig_mol = oechem.OEGraphMol()
prot_mol = oechem.OEGraphMol()
water_mol = oechem.OEGraphMol()
oth_mol = oechem.OEGraphMol()

## Make splitting split out covalent ligands
## TODO: look into different covalent-related options here
opts = oechem.OESplitMolComplexOptions()
opts.SetSplitCovalent(True)
opts.SetSplitCovalentCofactors(True)
print(
    oechem.OESplitMolComplex(lig_mol, prot_mol, water_mol, oth_mol, complex_mol)
)

print(
    complex_mol.NumAtoms(),
    lig_mol.NumAtoms(),
    prot_mol.NumAtoms(),
    water_mol.NumAtoms(),
    oth_mol.NumAtoms(),
)

True
2603 62 2256 24 261


In [24]:
## Load ref PDB molecule
ref_fn = "input_pdbs/reference.pdb"
ref_complex_mol = oechem.OEGraphMol()
ifs.open(ref_fn)
oechem.OEReadMolecule(ifs, ref_complex_mol)
ifs.close()
print(ref_complex_mol.NumAtoms())

## Perform splitting for ref
ref_lig_mol = oechem.OEGraphMol()
ref_prot_mol = oechem.OEGraphMol()
ref_water_mol = oechem.OEGraphMol()
ref_oth_mol = oechem.OEGraphMol()
print(
    oechem.OESplitMolComplex(
        ref_lig_mol, ref_prot_mol, ref_water_mol, ref_oth_mol, ref_complex_mol
    )
)
print(
    ref_complex_mol.NumAtoms(),
    ref_lig_mol.NumAtoms(),
    ref_prot_mol.NumAtoms(),
    ref_water_mol.NumAtoms(),
    ref_oth_mol.NumAtoms(),
)

2832
True
2832 34 2316 18 464


In [27]:
## Align new protein to ref
aln_res = oespruce.OESuperposeResults()
superpos = oespruce.OESuperpose()
superpos.SetupRef(ref_prot_mol)
superpos.Superpose(aln_res, prot_mol)
print(f"RMSD: {aln_res.GetRMSD()}")

prot_mol_aligned = prot_mol.CreateCopy()
print(aln_res.Transform(prot_mol_aligned))

## Save new aligned molecule for inspection
out_fn = "test/5WKK_split_aligned.pdb"
ofs = oechem.oemolostream()
ofs.SetFlavor(oechem.OEFormat_PDB, oechem.OEOFlavor_PDB_Default)
ofs.open(out_fn)
oechem.OEWriteMolecule(ofs, prot_mol_aligned)
ofs.close()

RMSD: 1.8288194400350355
True


# Load in new ligand and create DesignUnit

In [30]:
## Load ligand molecule
sdf_fn = f"fragalysis/aligned/Mpro-P2007_0A/Mpro-P2007_0A.sdf"
ifs = oechem.oemolistream()
ifs.SetFlavor(
    oechem.OEFormat_SDF,
    oechem.OEIFlavor_SDF_Default,
)
ifs.open(sdf_fn)
new_lig_mol = oechem.OEGraphMol()
oechem.OEReadMolecule(ifs, new_lig_mol)
ifs.close()
print(new_lig_mol.NumAtoms())

20


In [31]:
## Add explicit Hs to protein and new ligand to prep
prot_mol_prep = prot_mol_aligned.CreateCopy()
new_lig_prep = new_lig_mol.CreateCopy()
print(oechem.OEAddExplicitHydrogens(prot_mol_prep))
print(oechem.OEAddExplicitHydrogens(new_lig_prep))

True
True


In [34]:
## Make the design unit and save
du = oechem.OEDesignUnit()
print(oespruce.OEMakeDesignUnit(du, prot_mol_prep, new_lig_prep))
print(du.HasProtein(), du.HasLigand())
du_out_fn = 'test/5WKK_new_lig.oedu'
if du.HasProtein() and du.HasLigand():
    oechem.OEWriteDesignUnit(du_out_fn, du)

True
True True
